In [1]:
import pandas as pd 
#%pip install langchain_groq
from langchain_groq.chat_models import ChatGroq
import pandas as pd
import numpy as np
from itertools import product
from sklearn import tree
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix, recall_score
import ast
import time
import matplotlib.pyplot as plt

In [32]:
# Groq API and Models 
Groq_Token = open('..\\groqapi.txt', 'r').read().split("\n")[3]  # Do not share this key with anyone
groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}

In [3]:
# Loading the data
f = open("..\\HAR\\UCI HAR Dataset\\features.txt", 'r').read().split('\n')
feature_labels = pd.read_csv(r'../HAR/UCI HAR Dataset/features.txt',sep = '\\s+',header=None)
train_dataframeX = pd.read_csv(r'../HAR/UCI HAR Dataset/train/X_train.txt',sep = '\\s+',header=None)
train_dataframeY = pd.read_csv(r'../HAR/UCI HAR Dataset/train/y_train.txt',sep = '\\s+',header=None)
test_dataframeX = pd.read_csv(r'../HAR/UCI HAR Dataset/test/X_test.txt',sep = '\\s+',header=None)
test_dataframeY = pd.read_csv(r'../HAR/UCI HAR Dataset/test/y_test.txt',sep = '\\s+',header=None)
correlation_matrix = train_dataframeX.corr()

In [4]:
print("Feature labels"); display(feature_labels)
print("train_DataframeX"); display(train_dataframeX)
print("train_DataframeY");display(train_dataframeY)
print("test_DataframeX");display(test_dataframeX)
print("test_DataframeY");display(test_dataframeY)
print("correlation_matrix"); display(correlation_matrix)

Feature labels


,0,1
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z
3,4,tBodyAcc-std()-X
4,5,tBodyAcc-std()-Y
...,...,...
556,557,"angle(tBodyGyroMean,gravityMean)"
557,558,"angle(tBodyGyroJerkMean,gravityMean)"
558,559,"angle(X,gravityMean)"
559,560,"angle(Y,gravityMean)"


train_DataframeX


,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,0.404573,-0.117290,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,0.087753,-0.351471,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,0.299665,-0.057193,-0.181233,-0.195387,0.039905,0.077078,-0.282301,0.043616,0.060410,0.210795,...,-0.070157,-0.588433,-0.880324,-0.190437,0.829718,0.206972,-0.425619,-0.791883,0.238604,0.049819
7348,0.273853,-0.007749,-0.147468,-0.235309,0.004816,0.059280,-0.322552,-0.029456,0.080585,0.117440,...,0.165259,-0.390738,-0.680744,0.064907,0.875679,-0.879033,0.400219,-0.771840,0.252676,0.050053
7349,0.273387,-0.017011,-0.045022,-0.218218,-0.103822,0.274533,-0.304515,-0.098913,0.332584,0.043999,...,0.195034,0.025145,-0.304029,0.052806,-0.266724,0.864404,0.701169,-0.779133,0.249145,0.040811
7350,0.289654,-0.018843,-0.158281,-0.219139,-0.111412,0.268893,-0.310487,-0.068200,0.319473,0.101702,...,0.013865,0.063907,-0.344314,-0.101360,0.700740,0.936674,-0.589479,-0.785181,0.246432,0.025339


train_DataframeY


...

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,0.071645,-0.330370,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.401189,-0.121845,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,0.062891,-0.190422,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346
3,0.270298,-0.032614,-0.117520,-0.994743,-0.973268,-0.967091,-0.995274,-0.974471,-0.968897,-0.938610,...,0.116695,-0.344418,-0.736124,-0.017067,0.154438,0.340134,0.296407,-0.698954,0.284114,-0.077108
4,0.274833,-0.027848,-0.129527,-0.993852,-0.967445,-0.978295,-0.994111,-0.965953,-0.977346,-0.938610,...,-0.121711,-0.534685,-0.846595,-0.002223,-0.040046,0.736715,-0.118545,-0.692245,0.290722,-0.073857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,0.310155,-0.053391,-0.099109,-0.287866,-0.140589,-0.215088,-0.356083,-0.148775,-0.232057,0.185361,...,0.074472,-0.376278,-0.750809,-0.337422,0.346295,0.884904,-0.698885,-0.651732,0.274627,0.184784
2943,0.363385,-0.039214,-0.105915,-0.305388,0.028148,-0.196373,-0.373540,-0.030036,-0.270237,0.185361,...,0.101859,-0.320418,-0.700274,-0.736701,-0.372889,-0.657421,0.322549,-0.655181,0.273578,0.182412
2944,0.349966,0.030077,-0.115788,-0.329638,-0.042143,-0.250181,-0.388017,-0.133257,-0.347029,0.007471,...,-0.066249,-0.118854,-0.467179,-0.181560,0.088574,0.696663,0.363139,-0.655357,0.274479,0.181184
2945,0.237594,0.018467,-0.096499,-0.323114,-0.229775,-0.207574,-0.392380,-0.279610,-0.289477,0.007471,...,-0.046467,-0.205445,-0.617737,0.444558,-0.819188,0.929294,-0.008398,-0.659719,0.264782,0.187563


test_DataframeY


,0
0,5
1,5
2,5
3,5
4,5
...,...
2942,2
2943,2
2944,2
2945,2


correlation_matrix


,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,1.000000,0.148061,-0.256952,0.000619,-0.021903,-0.044617,0.006290,-0.022754,-0.047558,0.044062,...,0.030681,-0.017557,-0.015613,-0.544320,0.012173,0.037444,0.028844,-0.035257,0.034371,0.028242
1,0.148061,1.000000,-0.078769,-0.045160,-0.044920,-0.049746,-0.044180,-0.045049,-0.050402,-0.038108,...,-0.022395,-0.001587,-0.004459,0.070559,-0.013541,0.017967,0.075679,-0.005309,0.001053,-0.013903
2,-0.256952,-0.078769,1.000000,-0.020217,-0.016641,-0.008410,-0.018747,-0.015203,-0.001988,-0.037197,...,-0.020481,0.020091,0.019127,0.052841,-0.039836,-0.063609,-0.034037,0.008587,-0.015288,-0.022643
3,0.000619,-0.045160,-0.020217,1.000000,0.927461,0.851668,0.998632,0.920888,0.846392,0.980844,...,-0.065987,0.148034,0.115565,-0.035011,-0.021633,0.018985,-0.024810,-0.371653,0.471065,0.394825
4,-0.021903,-0.044920,-0.016641,0.927461,1.000000,0.895510,0.922803,0.997347,0.894509,0.917366,...,-0.105621,0.206227,0.176946,-0.020379,-0.012505,-0.008507,-0.014592,-0.380531,0.523600,0.433169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
556,0.037444,0.017967,-0.063609,0.018985,-0.008507,-0.018429,0.019389,-0.012546,-0.023525,0.025066,...,-0.026615,0.034514,0.024553,-0.006269,0.009141,1.000000,-0.116001,-0.005853,-0.012313,-0.019903
557,0.028844,0.075679,-0.034037,-0.024810,-0.014592,-0.006471,-0.024951,-0.012341,-0.007231,-0.028871,...,0.000102,-0.017937,-0.014865,-0.020823,0.035263,-0.116001,1.000000,0.023995,-0.005869,-0.005656
558,-0.035257,-0.005309,0.008587,-0.371653,-0.380531,-0.345011,-0.368191,-0.377025,-0.347389,-0.384192,...,0.087332,-0.086006,-0.079751,0.011880,0.023246,-0.005853,0.023995,1.000000,-0.783848,-0.643655
559,0.034371,0.001053,-0.015288,0.471065,0.523600,0.476006,0.466424,0.525081,0.477607,0.480229,...,-0.100125,0.086993,0.078079,0.001540,-0.012990,-0.012313,-0.005869,-0.783848,1.000000,0.594885


In [5]:
feature_labels

,0,1
0,1,tBodyAcc-mean()-X
1,2,tBodyAcc-mean()-Y
2,3,tBodyAcc-mean()-Z
3,4,tBodyAcc-std()-X
4,5,tBodyAcc-std()-Y
...,...,...
556,557,"angle(tBodyGyroMean,gravityMean)"
557,558,"angle(tBodyGyroJerkMean,gravityMean)"
558,559,"angle(X,gravityMean)"
559,560,"angle(Y,gravityMean)"


In [11]:
activity_dict = {1: 'WALKING', 2: 'WALKING_UPSTAIRS', 3: 'WALKING_DOWNSTAIRS', 4: 'SITTING', 5: 'STANDING', 6: 'LAYING'}

In [12]:
# Let us now change the feature labels to the actual feature names
feature_names = feature_labels[1].values.tolist()
train_dataframeX.columns = feature_names
test_dataframeX.columns = feature_names
# Changing the activity labels to the actual activity names
train_dataframeY.columns = ['Activity']
test_dataframeY.columns = ['Activity']
train_dataframeY['Activity'] = train_dataframeY['Activity'].map(activity_dict)
test_dataframeY['Activity'] = test_dataframeY['Activity'].map(activity_dict)

In [15]:
display(train_dataframeY)

,Activity
0,STANDING
1,STANDING
2,STANDING
3,STANDING
4,STANDING
...,...
7347,WALKING_UPSTAIRS
7348,WALKING_UPSTAIRS
7349,WALKING_UPSTAIRS
7350,WALKING_UPSTAIRS


In [16]:
train_dataframeY.values

array([['STANDING'],
       ['STANDING'],
       ['STANDING'],
       ...,
       ['WALKING_UPSTAIRS'],
       ['WALKING_UPSTAIRS'],
       ['WALKING_UPSTAIRS']], dtype=object)

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
scaler = StandardScaler()
X_scaled = scaler.fit_transform(train_dataframeX)
# Select top 60 features
selector = SelectKBest(score_func=f_classif, k=20)
X_reduced = selector.fit_transform(X_scaled, train_dataframeY.values.ravel())
selected_feature_indices = selector.get_support(indices=True)
selected_feature_names = train_dataframeX.columns[selected_feature_indices]
print("Selected features:", selected_feature_names)

Selected features: Index(['tBodyAcc-std()-X', 'tBodyAcc-max()-X', 'tBodyAcc-sma()',
       'tGravityAcc-mean()-X', 'tGravityAcc-max()-X', 'tGravityAcc-min()-X',
       'tGravityAcc-energy()-X', 'tBodyAccJerk-entropy()-X',
       'tBodyAccJerk-entropy()-Y', 'tBodyAccJerk-entropy()-Z',
       'tBodyAccJerkMag-entropy()', 'fBodyAcc-std()-X', 'fBodyAcc-mad()-X',
       'fBodyAcc-sma()', 'fBodyAcc-entropy()-X', 'fBodyAccJerk-entropy()-X',
       'fBodyAccJerk-entropy()-Y', 'fBodyAccJerk-entropy()-Z',
       'fBodyBodyAccJerkMag-entropy()', 'angle(X,gravityMean)'],
      dtype='object')


In [19]:
x_train_reduced = train_dataframeX[selected_feature_names]
x_test_reduced = test_dataframeX[selected_feature_names]
display(x_train_reduced); display(x_test_reduced)

,tBodyAcc-std()-X,tBodyAcc-max()-X,tBodyAcc-sma(),tGravityAcc-mean()-X,tGravityAcc-max()-X,tGravityAcc-min()-X,tGravityAcc-energy()-X,tBodyAccJerk-entropy()-X,tBodyAccJerk-entropy()-Y,tBodyAccJerk-entropy()-Z,tBodyAccJerkMag-entropy(),fBodyAcc-std()-X,fBodyAcc-mad()-X,fBodyAcc-sma(),fBodyAcc-entropy()-X,fBodyAccJerk-entropy()-X,fBodyAccJerk-entropy()-Y,fBodyAccJerk-entropy()-Z,fBodyBodyAccJerkMag-entropy(),"angle(X,gravityMean)"
0,-0.995279,-0.934724,-0.965523,0.963396,0.892055,0.977436,0.899469,-0.819949,-0.793046,-0.888853,-0.863415,-0.995422,-0.996889,-0.974733,-0.946357,-1.000000,-1.000000,-1.000000,-1.000000,-0.841247
1,-0.998245,-0.943068,-0.981930,0.966561,0.892060,0.984520,0.907829,-0.875096,-0.655362,-0.767381,-0.819928,-0.998680,-0.997890,-0.985812,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.844788
2,-0.995380,-0.938692,-0.983478,0.966878,0.892401,0.986770,0.908668,-0.753629,-0.673274,-0.747107,-0.794884,-0.996313,-0.994097,-0.985821,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.848933
3,-0.996091,-0.938692,-0.986093,0.967615,0.893817,0.986821,0.910621,-0.820804,-0.754968,-0.825279,-0.792321,-0.996312,-0.994547,-0.992812,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.848649
4,-0.998139,-0.942469,-0.992653,0.968224,0.893817,0.987434,0.912235,-0.850744,-0.746258,-0.796960,-0.850026,-0.998606,-0.997725,-0.992423,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-0.847865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7347,-0.195387,0.210795,0.000698,0.923148,0.867718,0.920554,0.794766,0.470793,0.605462,0.445862,0.603926,-0.221989,-0.050748,-0.028449,0.675244,0.546273,0.455341,0.173712,0.354983,-0.791883
7348,-0.235309,0.117440,-0.041310,0.918343,0.866116,0.920554,0.782407,0.534431,0.608132,0.552989,0.612144,-0.267430,-0.177661,-0.045731,0.608926,0.372223,0.357697,0.187539,0.361739,-0.771840
7349,-0.218218,0.043999,0.008034,0.919810,0.854641,0.933008,0.786085,0.657112,0.497936,0.512438,0.621533,-0.173212,-0.249486,-0.196488,0.530763,0.465329,0.422191,0.163524,0.189903,-0.779133
7350,-0.219139,0.101702,0.020011,0.922323,0.855988,0.933008,0.792538,0.586116,0.501824,0.379816,0.584586,-0.158192,-0.247028,-0.265529,0.402511,0.417495,0.346965,0.113978,0.240644,-0.785181


,tBodyAcc-std()-X,tBodyAcc-max()-X,tBodyAcc-sma(),tGravityAcc-mean()-X,tGravityAcc-max()-X,tGravityAcc-min()-X,tGravityAcc-energy()-X,tBodyAccJerk-entropy()-X,tBodyAccJerk-entropy()-Y,tBodyAccJerk-entropy()-Z,tBodyAccJerkMag-entropy(),fBodyAcc-std()-X,fBodyAcc-mad()-X,fBodyAcc-sma(),fBodyAcc-entropy()-X,fBodyAccJerk-entropy()-X,fBodyAccJerk-entropy()-Y,fBodyAccJerk-entropy()-Z,fBodyBodyAccJerkMag-entropy(),"angle(X,gravityMean)"
0,-0.938404,-0.894088,-0.877764,0.936489,0.906083,0.944461,0.829297,-0.085174,-0.310263,-0.510288,-0.129552,-0.948290,-0.930680,-0.865063,-0.339673,-0.470662,-0.672172,-0.596274,-0.484619,-0.720009
1,-0.975415,-0.894088,-0.969097,0.927404,0.856176,0.944461,0.805575,-0.497233,-0.655751,-0.614424,-0.527708,-0.984350,-0.975764,-0.957623,-0.604013,-0.606633,-1.000000,-0.942216,-0.723642,-0.698091
2,-0.993819,-0.939260,-0.976782,0.929915,0.856269,0.948704,0.812044,-0.699834,-0.674949,-0.582387,-0.633740,-0.994754,-0.993941,-0.977114,-0.946357,-1.000000,-0.939120,-0.864158,-0.898000,-0.702771
3,-0.994743,-0.938610,-0.974418,0.928881,0.856269,0.947309,0.809379,-0.699608,-0.614042,-0.607447,-0.638559,-0.995594,-0.994053,-0.978688,-1.000000,-1.000000,-1.000000,-0.900162,-1.000000,-0.698954
4,-0.993852,-0.938610,-0.975283,0.926600,0.853942,0.946221,0.803500,-0.744950,-0.637935,-0.661910,-0.764328,-0.994504,-0.993104,-0.982964,-0.946357,-1.000000,-0.939120,-1.000000,-1.000000,-0.692245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,-0.287866,0.185361,-0.197803,0.890968,0.823272,0.903413,0.712964,0.595651,0.525215,0.587045,0.579068,-0.269827,-0.240594,-0.201499,0.426656,0.439936,0.277066,0.337849,0.214449,-0.651732
2943,-0.305388,0.185361,-0.186842,0.891281,0.823272,0.903793,0.713748,0.684051,0.464601,0.528715,0.643984,-0.328225,-0.181938,-0.086811,0.611024,0.486474,0.394316,0.346014,0.267662,-0.655181
2944,-0.329638,0.007471,-0.232233,0.890897,0.822107,0.903793,0.712784,0.665375,0.530407,0.489835,0.600790,-0.316973,-0.301743,-0.181466,0.566462,0.406943,0.351133,0.071008,0.239540,-0.655357
2945,-0.323114,0.007471,-0.251372,0.892736,0.823679,0.909007,0.717394,0.687911,0.446009,0.423143,0.599803,-0.302423,-0.308029,-0.258653,0.496858,0.285182,0.182878,0.392158,0.200361,-0.659719


In [21]:
x_train_reduced["Activity"] = train_dataframeY["Activity"]
x_test_reduced["Activity"] = test_dataframeY["Activity"]

C:\Users\Rajeev Wankar\AppData\Local\Temp\ipykernel_23924\1677160487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_train_reduced["Activity"] = train_dataframeY["Activity"]
C:\Users\Rajeev Wankar\AppData\Local\Temp\ipykernel_23924\1677160487.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test_reduced["Activity"] = test_dataframeY["Activity"]


In [22]:
train_list = [x_train_reduced[x_train_reduced['Activity']==act].reset_index(drop=True) for act in activity_dict.values()]
test_list = [x_test_reduced[x_test_reduced['Activity'] == act].reset_index(drop=True) for act in activity_dict.values()]

In [37]:
# System Prompts
def infer(train_list, test_list,n_test, n_train, random_state=42, shuffle=False, delay=3):
    # n_test = 2 # Number of test samples per activity
    # n_train = 4 # Number of train samples per activity
    predicted_activities = []
    seed = random_state
    if shuffle:
        train_df = pd.concat([train_list[i].sample(n_train, random_state=seed) for i in range(6)], axis=0).sample(frac=1, random_state=seed).reset_index(drop=True)
    else:
        train_df = pd.concat([train_list[i].sample(n_train, random_state=seed) for i in range(6)], axis=0).reset_index(drop=True)
    test_df = pd.concat([test_list[i].sample(n_test, random_state=seed) for i in range(6)], axis=0).sample(frac=1, random_state=seed).reset_index(drop=True)
    for test in range(n_test*6):
        query = f"""
        * You are a Human Activity Recognition model.
        * Your task is to analyze the data given in the train data provided and guess the activity being performed by the person.
        * The activities are: WALKING, WALKING_UPSTAIRS, WALKING_DOWNSTAIRS, SITTING, STANDING, LAYING.
        * Provide a single label for the activity being performed by the person.
        * Feature names: {train_df.columns[:-1].to_list()}
        """
        for i in range(len(train_df)):
            query += f"* Train sample {i+1}: {train_df.iloc[i, :-1].values}\n"
            query += f"* Train label {i+1}: {train_df.iloc[i, -1]}\n\n"
        query += "Now, predict the activity for the following test sample:\n"
        query += f"Feature names: {train_df.columns[:-1].to_list()}\n"
        query += f"* Test sample: {test_df.iloc[test, :-1].values}\n"
        query += "Remember, just output a single label for the activity being performed. DO NOT say anything else."
        #To use Groq LLMs 
        model_name = "llama3.1-70b" # We can choose any model from the groq_models dictionary
        llm = ChatGroq(model=groq_models[model_name], api_key=Groq_Token, temperature=0)
        answer = llm.invoke(query)
        print(f"Test {test}: Predicted: {answer.content}, Actual: {test_df.iloc[test, -1]}")
        predicted_activities.append(answer.content)
        time.sleep(delay)
    acc=  accuracy_score(test_df["Activity"].to_list(), predicted_activities)
    return acc, predicted_activities

In [34]:
acc, predicted_activities = infer(train_list, test_list, 2, 4, random_state=42, shuffle=False, delay=3)

Test 0: Predicted: LAYING, Actual: LAYING
Test 1: Predicted: SITTING, Actual: STANDING
Test 2: Predicted: WALKING, Actual: WALKING
Test 3: Predicted: SITTING, Actual: STANDING
Test 4: Predicted: WALKING_UPSTAIRS, Actual: WALKING_DOWNSTAIRS
Test 5: Predicted: WALKING, Actual: WALKING_UPSTAIRS
Test 6: Predicted: WALKING, Actual: WALKING
Test 7: Predicted: LAYING, Actual: LAYING
Test 8: Predicted: WALKING_DOWNSTAIRS, Actual: WALKING_DOWNSTAIRS
Test 9: Predicted: SITTING, Actual: SITTING
Test 10: Predicted: WALKING, Actual: WALKING_UPSTAIRS
Test 11: Predicted: SITTING, Actual: SITTING


In [35]:
print(acc)

0.5833333333333334


In [38]:
acc, predicted_activities = infer(train_list, test_list, 2, 4, random_state=42, shuffle=True, delay=3)

Test 0: Predicted: LAYING, Actual: LAYING
Test 1: Predicted: SITTING, Actual: STANDING
Test 2: Predicted: WALKING, Actual: WALKING
Test 3: Predicted: STANDING, Actual: STANDING
Test 4: Predicted: WALKING, Actual: WALKING_DOWNSTAIRS
Test 5: Predicted: WALKING, Actual: WALKING_UPSTAIRS
Test 6: Predicted: WALKING, Actual: WALKING
Test 7: Predicted: LAYING, Actual: LAYING
Test 8: Predicted: WALKING_DOWNSTAIRS, Actual: WALKING_DOWNSTAIRS
Test 9: Predicted: STANDING, Actual: SITTING
Test 10: Predicted: WALKING_UPSTAIRS, Actual: WALKING_UPSTAIRS
Test 11: Predicted: SITTING, Actual: SITTING


In [39]:
print(acc)

0.6666666666666666


In [40]:
acc, predicted_activities = infer(train_list, test_list, 3, 5, random_state=40, shuffle=False, delay=3)

Test 0: Predicted: LAYING, Actual: LAYING
Test 1: Predicted: WALKING, Actual: WALKING
Test 2: Predicted: STANDING, Actual: SITTING
Test 3: Predicted: WALKING, Actual: WALKING_UPSTAIRS
Test 4: Predicted: LAYING, Actual: LAYING
Test 5: Predicted: SITTING, Actual: STANDING
Test 6: Predicted: STANDING, Actual: SITTING
Test 7: Predicted: STANDING, Actual: SITTING
Test 8: Predicted: WALKING, Actual: WALKING_UPSTAIRS
Test 9: Predicted: STANDING, Actual: STANDING
Test 10: Predicted: LAYING, Actual: LAYING
Test 11: Predicted: WALKING, Actual: WALKING
Test 12: Predicted: WALKING, Actual: WALKING
Test 13: Predicted: STANDING, Actual: STANDING
Test 14: Predicted: WALKING_DOWNSTAIRS, Actual: WALKING_DOWNSTAIRS
Test 15: Predicted: WALKING, Actual: WALKING_UPSTAIRS
Test 16: Predicted: WALKING_DOWNSTAIRS, Actual: WALKING_DOWNSTAIRS
Test 17: Predicted: WALKING_DOWNSTAIRS, Actual: WALKING_DOWNSTAIRS


In [41]:
print(acc)

0.6111111111111112
